In [22]:
# Imports for project

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

# Oscars Best Picture

In [23]:
# Import database
df = pd.read_csv('awards.csv')

# Drop unnecessary columns
df = df[['Award','Winner','Name','Film']]

df['Award'].unique()

array(['Actor', 'Actress', 'Art Direction', 'Cinematography',
       'Directing (Comedy Picture)', 'Directing (Dramatic Picture)',
       'Engineering Effects', 'Outstanding Picture',
       'Unique and Artistic Picture', 'Writing (Adaptation)',
       'Writing (Original Story)', 'Writing (Title Writing)',
       'Special Award', 'Directing', 'Writing', 'Outstanding Production',
       'Sound Recording', 'Short Subject (Cartoon)',
       'Short Subject (Comedy)', 'Short Subject (Novelty)',
       'Assistant Director', 'Film Editing', 'Music (Scoring)',
       'Music (Song)', 'Dance Direction', 'Writing (Screenplay)',
       'Actor in a Supporting Role', 'Actress in a Supporting Role',
       'Short Subject (Color)', 'Short Subject (One Reel)',
       'Short Subject (Two Reel)', 'Irving G. Thalberg Memorial Award',
       'Music (Original Score)', 'Cinematography (Black and White)',
       'Cinematography (Color)', 'Special Effects',
       'Art Direction (Black and White)', 'Art Direct

In [24]:
# Replace all the variant names of Best Picture into Best Picture
bp_variants = {'Outstanding Picture', 'Outstanding Production', 'Outstanding Motion Picture', 'Best Motion Picture', 'Best Picture'}

for index, row in df.iterrows():
    if(row['Award'] in bp_variants):
        df.loc[index,'Award'] = 'Best Picture'

In [25]:
# Only keep best picture awards
df = df[df['Award']=='Best Picture']

In [26]:
print((df[df['Winner'] == 1]).to_string())

             Award  Winner                                              Name                                               Film
21    Best Picture     1.0                            Paramount Famous Lasky                                              Wings
64    Best Picture     1.0                               Metro-Goldwyn-Mayer                                The Broadway Melody
100   Best Picture     1.0                   All Quiet on the Western Front                                           Universal
140   Best Picture     1.0                                         Cimarron                                           RKO Radio
178   Best Picture     1.0                                      Grand Hotel                                 Metro-Goldwyn-Mayer
237   Best Picture     1.0                                        Cavalcade                                                 Fox
301   Best Picture     1.0                            It Happened One Night                             

We noticed that there are 89 pictures instead of the expected 88. We went through them by hand and found that M*A*S*H was put under winning when it should not have. Also there are movies in which the producer is mistakenly put as film name.

In [27]:
# M*A*S*H was not a winner
df.loc[4852,'Winner']=np.nan
# Change to correct film names
df.loc[21,'Name']="Wings"
df.loc[64,'Name']="The Broadway Melody"

Several of the nominations had wrong names too...

In [28]:
df.loc[19,'Name']="The Racket"
df.loc[20,'Name']="7th Heaven"
df.loc[62,'Name']="Alibi"
df.loc[63,'Name']="In Old Arizona"
df.loc[65,'Name']="Hollywood Revue"
df.loc[66,'Name']="The Patriot"

In [29]:
# Now we can drop the name row to only have film name
df = df[['Name','Winner']]

In [30]:
df['Name'].unique().size

523

In [31]:
df['Name'].value_counts()

Romeo and Juliet                                                         2
Cleopatra                                                                2
Heaven Can Wait                                                          2
Moulin Rouge                                                             2
Mutiny on the Bounty                                                     2
Secrets & Lies                                                           1
Hope and Glory                                                           1
Barry Lyndon                                                             1
Selma                                                                    1
12 Years a Slave                                                         1
Mildred Pierce                                                           1
Wake Island                                                              1
Elmer Gantry                                                             1
Star Wars                

This is because there are movies of the same name that got nominated/won for Best Picture in different years! But that's OK because it's just 5 entires out of 528!!

In [32]:
# Strip whitespace and lowercase the movie titles for consistency
df['Name']=df['Name'].str.strip()
df['Name']=df['Name'].str.lower()
df['Winner'].fillna(0,inplace=True)

In [33]:
df['Nominated']=1

In [34]:
df

,Name,Winner,Nominated
19,the racket,0.0,1
20,7th heaven,0.0,1
21,wings,1.0,1
62,alibi,0.0,1
63,in old arizona,0.0,1
64,the broadway melody,1.0,1
65,hollywood revue,0.0,1
66,the patriot,0.0,1
100,all quiet on the western front,1.0,1
101,the big house,0.0,1


# Month of Release

In [35]:
dfm = pd.read_csv('the-movies-dataset/movies_metadata.csv')
dfm.dropna(subset=['release_date'], inplace = True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
def replace(date):
    output = date[5:7]
    return output



dfm['release_date']=dfm['release_date'].apply(replace)
dfm=dfm[["title","imdb_id","release_date"]]

In [37]:
# Strip whitespace and tolower for consistency
dfm["title"]=dfm["title"].str.strip()
dfm["title"]=dfm["title"].str.lower()

In [38]:
# There seems to be 3 movies with blank months. Remove them!
dfm['release_date'].value_counts()

01    5912
09    4838
10    4615
12    3786
11    3661
03    3553
04    3453
08    3394
05    3339
06    3153
02    3032
07    2640
         3
Name: release_date, dtype: int64

In [39]:
# Remove all rows with blank release dates
dfm = dfm[dfm["release_date"]!=""]

In [40]:
# Set all month columns as 0 by default
dfm['Jan-Feb']=0
dfm['Mar-Apr']=0
dfm['May-Jun']=0
dfm['Jul-Aug']=0
dfm['Sept-Oct']=0
dfm['Nov-Dec']=0

# Set each month category with repective truth value
for index,row in dfm.iterrows():
    month_num = int(row['release_date'])
    if(month_num <=2):
        dfm.loc[index,'Jan-Feb']=1
    elif(month_num<=4):
        dfm.loc[index,'Mar-Apr']=1
    elif(month_num<=6):
        dfm.loc[index,'May-Jun']=1
    elif(month_num<=8):
        dfm.loc[index,'Jul-Aug']=1
    elif(month_num<=10):
        dfm.loc[index,'Sept-Oct']=1
    elif(month_num<=12):
        dfm.loc[index,'Nov-Dec']=1

In [41]:
dfm

,title,imdb_id,release_date,Jan-Feb,Mar-Apr,May-Jun,Jul-Aug,Sept-Oct,Nov-Dec
0,toy story,tt0114709,10,0,0,0,0,1,0
1,jumanji,tt0113497,12,0,0,0,0,0,1
2,grumpier old men,tt0113228,12,0,0,0,0,0,1
3,waiting to exhale,tt0114885,12,0,0,0,0,0,1
4,father of the bride part ii,tt0113041,02,1,0,0,0,0,0
5,heat,tt0113277,12,0,0,0,0,0,1
6,sabrina,tt0114319,12,0,0,0,0,0,1
7,tom and huck,tt0112302,12,0,0,0,0,0,1
8,sudden death,tt0114576,12,0,0,0,0,0,1
9,goldeneye,tt0113189,11,0,0,0,0,0,1


# IMDB Weighted Ratings

In [44]:
# Renamed data from title.ratings.tsv.gz->data.tsv
ratings = pd.read_csv('the-movies-dataset/ratings.tsv',delimiter='\t')
# Renamed data from title.basics.tsv.gz->data.tsv
basics = pd.read_csv('the-movies-dataset/basics.tsv', delimiter='\t')

FileNotFoundError: File b'the-movies-dataset/ratings.tsv' does not exist

In [ ]:
# Remove all things that arent movies
basics = basics[basics['titleType'] == "movie"]
# Remove all adult titles
basics = basics[basics['isAdult'] == 0]
# Remove unnecessary columns
basics = basics.drop(['titleType','originalTitle','isAdult','startYear','endYear','runtimeMinutes','genres'],axis=1)

In [ ]:
# Do a inner join on movies that have both ratings and basic info
df = pd.merge(basics, ratings, on='tconst', how='inner')
# Dont drop the IMDB id just in case for matching
#df = df.drop(['tconst'], axis=1)

In [ ]:
# Drop movies with too little votes
# df = df[df['numVotes']>50000]
# Add a new column with weighted ratings based on minimum votes
minVote = 30000

# Scroll down to the bottom of the following link to check how the weighted rating was calculated
# https://help.imdb.com/article/imdb/track-movies-tv/faq-for-imdb-ratings/G67Y87TFYYP6TWAV?ref_=helpsect_pro_2_4#
df = df.assign(weighted_ratings=((df['numVotes']/(df['numVotes']+minVote))*df['averageRating'])+(minVote/(df['numVotes']+minVote))*df['averageRating'].mean())

df.sort_values(by=['weighted_ratings'], ascending=False)

# Production Awards

In [46]:
awards_df = pd.read_csv("awards.csv")

# For some reason the dataset switched the name/film columns starting at the 3rd ceremony
# We had to compensate for this by moving the name of the film to the correct column for the first 3 ceremonies
for index, row in awards_df.iterrows():
    if(row["Ceremony"] < 3):
        awards_df.set_value(index,'Name',row["Film"])


# Drop the Ceremony and Year category since they aren't important
awards_df.drop(["Ceremony", "Year", "Film"], axis=1, inplace=True)
awards_df.rename(columns={'Name': 'Film'}, inplace=True)
# Strip whitespace and tolower for consistency
awards_df["Film"]=awards_df["Film"].str.strip()
awards_df["Film"]=awards_df["Film"].str.lower()

awards_df = awards_df.fillna(0)



awards = ["Film Editing", "Cinematography", "Makeup", "Production Design", "Art Direction",
         "Sound Editing", "Sound Mixing", "Special Effects" "Special Visual Effects", 
          "Special Achievement Award (Visual Effects)", "Visual Effects", "Engineering Effects"]


# Remove all of the rows that do not pertain to any of the production related awards
awards_df = awards_df[awards_df["Award"].isin(awards)]

# Number of nominations for production related awards 
nominations = awards_df["Film"].value_counts()

# Remove the nominations and only get the winenrs
# awards_df = awards_df[awards_df["Winner"] == 1]

# Numer of winners for production related awards
winners_count = awards_df["Film"].value_counts()



In [47]:
# Make a new DF indexed by film name
prod_awards=pd.DataFrame(columns=['Film','Nominated Production',"Film Editing", "Cinematography", "Makeup", "Production Design", "Art Direction",
         "Sound Editing", "Sound Mixing", "Special Effects" "Special Visual Effects", 
          "Special Achievement Award (Visual Effects)", "Visual Effects", "Engineering Effects"])

In [48]:
prod_awards['Film'] = awards_df['Film'].unique()
# Initialize all fields to 0
prod_awards=prod_awards.fillna(0)
prod_awards

,Film,Nominated Production,Film Editing,Cinematography,Makeup,Production Design,Art Direction,Sound Editing,Sound Mixing,Special EffectsSpecial Visual Effects,Special Achievement Award (Visual Effects),Visual Effects,Engineering Effects
0,sunrise,0,0,0,0,0,0,0,0,0,0,0,0
1,the dove; tempest,0,0,0,0,0,0,0,0,0,0,0,0
2,7th heaven,0,0,0,0,0,0,0,0,0,0,0,0
3,the devil dancer; the magic flame; sadie thompson,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0
5,wings,0,0,0,0,0,0,0,0,0,0,0,0
6,the patriot,0,0,0,0,0,0,0,0,0,0,0,0
7,the bridge of san luis rey,0,0,0,0,0,0,0,0,0,0,0,0
8,dynamite,0,0,0,0,0,0,0,0,0,0,0,0
9,alibi; and the awakening,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
# Removing specific movie names that put more than one movie in the same name
prod_awards = prod_awards[prod_awards['Film']!="the devil dancer; the magic flame; sadie thompson"]
prod_awards = prod_awards[prod_awards['Film']!="the dove; tempest"]
prod_awards = prod_awards[prod_awards['Film']!="alibi; and the awakening"]
prod_awards = prod_awards[prod_awards['Film']!="four devils; and street angel"]
# Removing specific movie names that put more than one movie in the same name
awards_df = awards_df[awards_df['Film']!="the devil dancer; the magic flame; sadie thompson"]
awards_df = awards_df[awards_df['Film']!="the dove; tempest"]
awards_df = awards_df[awards_df['Film']!="alibi; and the awakening"]
awards_df = awards_df[awards_df['Film']!="four devils; and street angel"]

In [50]:
# Manually insert all the names with multiple production awards
prod_awards.loc[884,"Film"]="the dove"
prod_awards.loc[885,"Film"]="tempest"
prod_awards.loc[886,"Film"]="the devil dancer"
prod_awards.loc[887,"Film"]="the magic flame"
prod_awards.loc[888,"Film"]="saddie thompson"
prod_awards.loc[889,"Film"]="alibi"
prod_awards.loc[890,"Film"]="the awakening"
prod_awards.loc[891,"Film"]="four devils"
prod_awards=prod_awards.fillna(0)
# Every film was nominated
prod_awards["Nominated Production"]=1

# Set the awards for these two films that won
prod_awards.loc[884,"Art Direction"]=1.0
prod_awards.loc[885,"Art Direction"]=1.0

In [57]:
# Temporarily set index based on film name
prod_awards.set_index('Film',inplace=True)
prod_awards

,index,Nominated Production,Film Editing,Cinematography,Makeup,Production Design,Art Direction,Sound Editing,Sound Mixing,Special EffectsSpecial Visual Effects,Special Achievement Award (Visual Effects),Visual Effects,Engineering Effects
Film,,,,,,,,,,,,,
sunrise,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7th heaven,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wings,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
the patriot,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the bridge of san luis rey,5,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
dynamite,6,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
street angel,7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
our dancing daughters,8,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
for index,row in awards_df.iterrows():
    if row['Award']=='Film Editing':
            prod_awards.loc[row["Film"],'Film Editing']=row["Winner"]
    elif row['Award']=='Art Direction':
            prod_awards.loc[row["Film"],'Art Direction']=row['Winner']
    elif row['Award']=='Cinematography':
            prod_awards.loc[row["Film"],'Cinematography']=row['Winner']
    elif row['Award']=='Visual Effects':
            prod_awards.loc[row["Film"],'Visual Effects']=row['Winner']
    elif row['Award']=='Makeup':
            prod_awards.loc[row["Film"],'Makeup']=row['Winner']
    elif row['Award']=='Sound Editing':
            prod_awards.loc[row["Film"],'Sound Editing']=row['Winner']
    elif row['Award']=='Sound Mixing':
            prod_awards.loc[row["Film"],'Sound Mixing']=row['Winner']
    elif row['Award']=='Production Design':
            prod_awards.loc[row["Film"],'Production Design']=row['Winner']
    elif row['Award']=='Special Achievement Award (Visual Effects)':
            prod_awards.loc[row["Film"],'Special Achievement Award (Visual Effects)']=row['Winner']
    elif row['Award']=='Engineering Effects':
            prod_awards.loc[row["Film"],'Engineering Effects']=row['Winner']

In [56]:
# Set index back to numbers and not film name
prod_awards=prod_awards.reset_index()
prod_awards

,index,Film,Nominated Production,Film Editing,Cinematography,Makeup,Production Design,Art Direction,Sound Editing,Sound Mixing,Special EffectsSpecial Visual Effects,Special Achievement Award (Visual Effects),Visual Effects,Engineering Effects
0,0,sunrise,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,7th heaven,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,wings,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,the patriot,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,the bridge of san luis rey,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,dynamite,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,street angel,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,our dancing daughters,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,white shadows in the south seas,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Actor Awards

In [61]:
# Setting up the data frame to contain acting related awards

actors_df = pd.read_csv('awards.csv')

actors_df = actors_df.loc[((actors_df['Award'] == "Actor") | (actors_df['Award'] == "Actress") | (actors_df['Award'] == "Actor in a Supporting Role") | (actors_df['Award'] == "Actress in a Supporting Role"))]

actors_df = actors_df.drop(["Name", "Year", "Ceremony"], axis=1)
actors_df.fillna(0, inplace=True)

In [62]:
awards_df = pd.DataFrame(columns=["Film", "Nominated Actor", "Actor", "Actress", "Supporting Actor", "Supporting Actress"])
awards_df["Film"] = actors_df["Film"].unique()
awards_df["Nominated Actor"] = 1
awards_df.fillna(0, inplace=True)

In [64]:
actors_df.set_index('Film',inplace=True)
awards_df.set_index('Film',inplace=True)

for index, row in actors_df.iterrows():
    if(row["Award"] == "Actor"):
        awards_df.loc[index, "Actor"] = row["Winner"]
    elif(row["Award"] == "Actress"):
        awards_df.loc[index, "Actor"] = row["Winner"]
    elif(row["Award"] == "Supporting Actor"):
        awards_df.loc[index, "Actor"] = row["Winner"]
    elif(row["Award"] == "Support Actress"):
        awards_df.loc[index, "Actor"] = row["Winner"]

actors_df.reset_index(inplace=True)
awards_df.reset_index(inplace=True)

awards_df

,Film,Nominated Actor,Actor,Actress,Supporting Actor,Supporting Actress
0,The Noose,1,0.0,0,0,0
1,The Last Command,1,1.0,0,0,0
2,A Ship Comes In,1,0.0,0,0,0
3,7th Heaven,1,1.0,0,0,0
4,Sadie Thompson,1,0.0,0,0,0
5,Thunderbolt,1,0.0,0,0,0
6,In Old Arizona,1,1.0,0,0,0
7,Alibi,1,0.0,0,0,0
8,The Valiant,1,0.0,0,0,0
9,The Patriot,1,0.0,0,0,0
